Code from https://github.com/pipidog/keras_to_tensorflow/blob/master/keras_to_tensorflow.py

In [ ]:
# This file shows how to save a keras model to tensorflow pb file 
# and how to use tensorflow to reload the model and inferece by the model 

from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import numpy as np
import os
from keras.models import model_from_json

np.random.seed(0)

# parameter ==========================
wkdir = os.path.abspath(os.getcwd())
pb_filename = 'lego_model.pb'

# build a keras model ================
# load json and create model
json_file = open('lego_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("lego_weights.h5")

model.compile(loss='categorical_crossentropy',
                     optimizer='rmsprop',
                     metrics=['accuracy']) 

# save model to pb ====================
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph

# save keras model as tf pb files ===============
from keras import backend as K
frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])
tf.train.write_graph(frozen_graph, wkdir, pb_filename, as_text=False)
